In [16]:
from abc import ABC, abstractmethod
import json


class Item(ABC):
    def __init__(self, title):
        self.title = title
        self._checked_out = False

    @property
    def is_checked_out(self) -> bool:
        return self._checked_out

    def checkout(self):
        self._checked_out = True

    def return_item(self):
        self._checked_out = False

    @abstractmethod
    def __str__(self):
        pass


class Book(Item):
    def __init__(self, title, author, isbn):
        super().__init__(title)
        self.author = author
        self.isbn = isbn

    def __str__(self):
        return f"Book: {self.title} by {self.author}"


class Magazine(Item):
    def __init__(self, title, volume, issue_number):
        super().__init__(title)
        self.volume = volume
        self.issue_number = issue_number

    def __str__(self):
        return f"Magazine: {self.title}, Vol. {self.volume}, Issue {self.issue_number}"


class ItemNotFoundError(Exception):
    pass


class ItemNotAvailableError(Exception):
    pass


class ItemNotCheckedOutError(Exception):
    pass


class Library:
    def __init__(self):
        self.items = []

    def add_item(self, item):
        self.items.append(item)

    def get_books(self):
        return [item for item in self.items if isinstance(item, Book)]

    def get_magazines(self):
        return [item for item in self.items if isinstance(item, Magazine)]

    def checkout(self, title: str):
        found = False
        for item in self.items:
            if item.title == title:
                found = True
                if item.is_checked_out:
                    raise ItemNotAvailableError
                else:
                    item.checkout()
                    return
        if not found:
            raise ItemNotFoundError

    def return_item(self, title: str):
        found = False
        for item in self.items:
            if item.title == title:
                found = True
                if not item.is_checked_out:
                    raise ItemNotCheckedOutError
                else:
                    item.return_item()
                    return
        if not found:
            raise ItemNotFoundError

    @classmethod
    def load_from_file(cls, filename: str):
        library = cls()
        with open(filename, 'r') as file:
            data = json.load(file)
            for item_data in data['books']:
                library.add_item(Book(**item_data))
            for item_data in data['magazines']:
                library.add_item(Magazine(**item_data))
        return library

    def __iter__(self):
        self._index = 0
        return self

    def __next__(self):
        if self._index < len(self.items):
            item = self.items[self._index]
            self._index += 1
            return item
        else:
            raise StopIteration        